# **この Notebook は、[Google Colab: PDF to CSV 変換器を Colab に設置 \[第二話 死闘篇\]  – NaN は dtype: float で捕捉！）](https://ggcs.io/2020/08/11/google-colab-pdf-export-02/)の手順詳細版です。**

- Website: ごたごた気流調査所  https://ggcs.io
- GitHub : Gota Gota Current Survey  https://github.com/ggcurrs
- Version 1.2.0
- Date   : 2020-08-11
- Updated: 2020-08-17

# 作業準備


- 前提1: tabula-py が Google Drive の ~/My Drive/Colab Notebooks/my-modules にインストールされていること。
- 前提2: 読み込み元の PDF が、Google Drive の ~/My Drive/pdf_project/data/2018_co_list_jp_r.pdf に置かれていること。
もし上記の前提が満たされていないようでしたら、[Google Colab: PDF to CSV 変換器を Colab に設置 \[第一話 立志篇\] – これでコピペ作業 から開放！](https://ggcs.io/2020/08/05/google-colab-pdf-export-01/)を参照の上、準備を整えてください。

In [ ]:
# Google Drive のマウント。
from google.colab import drive
drive.mount('/content/drive')

# Modules の import.
import os
import sys
# PATH を通す（Python に modules の場所を教える）
MODULE_PATH = '/content/drive/My Drive/Colab Notebooks/my-modules'
sys.path.append(MODULE_PATH)
import tabula  # Module の場所を教えたので、import.
import pandas as pd  # 念のため明示的に import しておく。


# ディレクトリ構造を定義する。
PROJECT_ROOT_PATH = '/content/drive/My Drive/pdf_project'
DATA_PATH   = os.path.join(PROJECT_ROOT_PATH, 'data')
OUTPUT_PATH = os.path.join(PROJECT_ROOT_PATH, 'output')

print('準備完了 🍻')

# PDF ファイルの読み込み (Load a PDF File)

In [ ]:
%%time
# PDF ファイルの読み込み（30 秒程度掛かります）
# WARNING が出ますが、今回の目的との関係では無視して差し支えありません。
df_list = tabula.read_pdf(os.path.join(DATA_PATH, '2018_co_list_jp_r.pdf'), pages='5-112')

# Data Cleansing

## **現状確認**
本番作業は Column > Rows の順番で行いますが、作業の都合上、DataFrame （以下「DF」）の shape を先に確認します。

### DataFrame Shape の確認
- tabula で読み込んだ PDF ファイルには、PDF のページで p.5 から p.112 までの合計 108 pages 分の DF がリストの形で格納されています。
- ここで、それぞれの DF の shape （row \* columns）を確認してみます（TABLE 1）。
1. すべての DF が 6 columns なので、これは吉報。項目列に大きな問題はなさそうです。
2. ほとんどの DF が 48 rows（p.112 (6, 6) は最後の表だから 6 rows で OK）で、オリジナル PDF と比較してもそうなっているのでこれも良し 👍
3. しかし、たとえば p.012 (111, 6) （= df_list[7]）などの raw 数は明らかにおかしい😱
  - のでオリジナル PDF と比較するなどして確認すると…。
  - これはマズイ。恐らくおおもとの原稿となった Excel でセル内改行（Option+改行）が行われていたため、本来は 1 row と解釈すべきところ、tabula が 3 rows と解釈してしまった模様。

In [ ]:
# TABLE 1: Shape of DFs
for i, each_df in enumerate(df_list):
  if i % 5 == 0:
    print()
  print('p.'+str(i+5).zfill(3), each_df.shape, ' ', end='')


### Column Names の確認
- 次のスクリプトを走らせて 108 個の DF の 表頭（Microsoft Word でいうところの「タイトル行」）を確認すると以下のことがわかります（TABLE 2）。
1. 異なる表頭（column names）が混在している。これは後で DF を 1 個にまとめる（concatenate）ときに悲劇を招く原因となる（オリジナル PDF の column names は揃っていたのだが tabula 読込 / parse 時に乱れてしまった ）。
2. Column name に全角文字やスペース、改行文字（\r）などが含まれている。作業の都合から言うと、これを放置しておくとあとあと悩みのタネになる。

In [ ]:
# TABLE 2: Column Names of the DFs
# オリジナルの Column names 確認
column_names = [each_df.columns for each_df in df_list]
column_names

## 方針

### Column 関係
- 今後のこと（Pandas や database で取り扱うこと）を考えると、今のうちに column names は以下のルールで付け替えておいた方が無難です。
  - 半角英数字（acsii 文字）のみ使用
  - space やヒトクセありそうな記号類は使わない
  - 英字は原則として小文字とする

### Row 関係
  - 問題が起きているところは概ね、3 行で 1 組（triad）になっている印象（TABLE 3）。
  - たとえば、index =  [6, 7, 8] の triad を例に見ると、
row 7 の column 0 （'州' ただし次 step で 'state_code' に rename）には value ('DL') が入っているが、その上下の row 6  と row 8 が NaN になっている（NaN-str-NaN pattern）。
  - 他の部分を見ても同様なので、このパターンになっている 3-rows をワンセット（triad）にして片付ければなんとかなりそう。
  - どの column を対象に「Triad 狩り」をするかも重要。実際、column 0 を対象にした場合は OK だが、column 2 を対象に狩を行うと、無実の row （たとえば index = 12 の row など）まで捕捉して変換してしまって詰みます😱
  - さらに、この Triad pattern が予期しない rows を捕捉していないかも確かめておく必要がある。
  - あ！ index =  [36, 37, 38] は triad として捕捉してしまうとマズイ。このままでは bug です（が、見切り発車します 🚃）。

In [ ]:
# TABLE 3: 'TRIADs' found in DF.
df_list[7].head(39)

## 実験 (Test)

### Column Names の変更実験
- Column names の変更の方は単純な話なので、こちらを先に片付けることにします（結果 > TABLE 4）。
- 名前の対応は、
  - 州: state_code
  - 拠点地名: location
  - 番号: sc_num
  - インド進出企業名: 'com_in'
  - 親会社名: com_jp
  - 業種: biz_type

In [ ]:
# オリジナルの DF に影響を与えないよう、念のためコピーに対して操作
test_df_list = df_list.copy()

In [ ]:
# 新しい column names の定義
new_column_names = ['state_code', 'location', 'sc_num', 'com_in', 'com_jp', 'biz_type']
# 108 個の DataFrame の column name を統一する。
for each_df in test_df_list:
  each_df.columns = new_column_names

In [ ]:
# TABLE 4: Renamed Columns
# 変換後の Column names 確認
for each_df in test_df_list:
  print(each_df.columns)

### Non-48-row-DF の整形実験
1. 整形操作概要
- 上記の「現状確認」を眺めてみると、columns 3, 4, 5 のそれぞれについて、NaN-str-NaN パターンの 3 rows を 1 row にまとめ、かつ、文字列 'nan' が入らないようにすればよさそう。方法としては;
  - Method A: リスト内包表記を使う方法;
  - Method B: Filter を使う方法 1;
  - Method C: Filter を使う方法 2

  で試してみる。
2. 結果を見ると、いずれの Method でも得られる結果に違いはありませんが、別途測定したところ Method A（リスト内包表記）の処理速度が若干早かったので、本番では Method A を使うことにします。

In [ ]:
# Methods A, B and C を試してみる。
def akb_48_test(bad_df):
  '''Search for the NaN-str-NaN pattern in the column 2, 
     educate and transform non_48 DataFrames to akb_48 DataFrames'''
  bad_df.reset_index()  # Preparation / 念のため
  for i in range(1, len(bad_df) - 1):
    # Find NaN-str-NaN (float-str-float) patterns ('triad' streaches)
    # in the column 2.
    if (
        (type(bad_df.iat[i-1, 0]) == float) &
        (type(bad_df.iat[i  , 0]) == str) &
        (type(bad_df.iat[i+1, 0]) == float)):
      # Target cells in the columns 3, 4, and 5.
      clm_3_triad = [bad_df.iat[i-1, 3], bad_df.iat[i, 3], bad_df.iat[i+1, 3]]
      clm_4_triad = [bad_df.iat[i-1, 4], bad_df.iat[i, 4], bad_df.iat[i+1, 4]]
      clm_5_triad = [bad_df.iat[i-1, 5], bad_df.iat[i, 5], bad_df.iat[i+1, 5]]

      # clm は修正前、new は修正したもの。並べて比較
      print('\nIndex', str(i).zfill(3), '± 1',
            
            '\nclm_3_triad', clm_3_triad,  # Method A (List comprehension)
            '\nnew_3_triad', [x for x in clm_3_triad if not type(x) == float],

            '\n\nclm_4_triad', clm_4_triad,  # Method B (Filter/lambda)
            '\nnew_4_triad', list(filter(lambda x: not type(x) == float, 
                                         clm_4_triad)), 

            '\n\nclm_5_triad', clm_5_triad,  # Method C (Filter/lambda)
            '\nnew_5_triad', list(filter(lambda x: x if not type(x) == float 
                                         else None, clm_5_triad)),'\n'
            )
      

if __name__ == '__main__':
  akb_48_test(test_df_list[7])

## 本番

### Column Names の変更
- 上記の「実験」で行なったのと全く同じ操作を本番用 DF 全体（108 個）に対して行います。

In [ ]:
# Rename columns.
new_column_names = ['state_code', 'location', 'sc_num', 'com_in', 'com_jp', 'biz_type']
# 108 個の DataFrame の column name を統一する。
for each_df in df_list:
  each_df.columns = new_column_names

### Non-48-row-DF の整形
- 108 個の DF のうち、ほとんど（104 個）は 48 rows \* 6 columns (最後の DF は 6 rows \* 6 columns）なので、下手に整形操作を加えて壊しちゃうと嫌過ぎる。
- そこで、問題のある 4 個の DF を含んだリストを隔離（list_df_non_48）し、これに対してだけ修正操作を行うことにします。

In [ ]:
# Isolate non_48-row type DFs as df_non_48.
list_df_non_48 = [each_df for each_df in df_list if len(each_df) > 48]
list_df_48     = [each_df for each_df in df_list if len(each_df) <= 48]

In [ ]:
def akb_48(bad_df):
  '''Educate and transform non_48 DataFrames to akb_48 DataFrames'''
  # Create 'good_df' for output.
  good_df = bad_df.copy()
  bad_df.reset_index()  # Preparation / 念のため
  for i in range(1, len(bad_df) - 1):
    # Find NaN-str-NaN (float-str-float) patterns ('triad' streaches)
    # in the column 0.
    if (
        (type(bad_df.iat[i-1, 0]) == float) &
        (type(bad_df.iat[i  , 0]) == str) &
        (type(bad_df.iat[i+1, 0]) == float)):
      # Target triads in the columns (clm) 3, 4, and 5
      clm_3_triad = [bad_df.iat[i-1, 3], bad_df.iat[i, 3], bad_df.iat[i+1, 3]]
      clm_4_triad = [bad_df.iat[i-1, 4], bad_df.iat[i, 4], bad_df.iat[i+1, 4]]
      clm_5_triad = [bad_df.iat[i-1, 5], bad_df.iat[i, 5], bad_df.iat[i+1, 5]]
      # Merge each triad into one, remove NaN (NaN: dtype = float).
      new_clm_3 = ', '.join([x for x in clm_3_triad if not type(x) == float])
      new_clm_4 = ', '.join([x for x in clm_4_triad if not type(x) == float])
      new_clm_5 = ', '.join([x for x in clm_5_triad if not type(x) == float])
      # Refresh the columns 3, 4, 5 of the good_df (output DF).
      good_df.iat[i, 3] = new_clm_3
      good_df.iat[i, 4] = new_clm_4
      good_df.iat[i, 5] = new_clm_5

  # Drop gargabe rows
  good_df.dropna(subset= ['sc_num'], inplace=True)
  # After dropping rows, reset the index!
  good_df.reset_index(drop=True, inplace=True)
  
  return good_df


if __name__ == '__main__':
  # Create an empty list.
  list_df_yes_48 = []
  # Cleansing
  for bad_df in list_df_non_48:
    good_df = akb_48(bad_df)
    list_df_yes_48.append(good_df)


### 結果確認
- ぱっと見は良い（概ねうまくいった）のだけど、
- 以下の rows に問題がある。 > 最後に目視確認して手作業で修正した方が早いので、そのようにする方針（敗北感…😭）。
1. list_df_new[0]
  - 13	HR	Bawal	7
  - 18	HR	Bawal	12
  - 32	HR	Bawal	26
  - 34	HR	Bawal	28
  - 41	HR	Dharuhera	35
2. list_df_new[1]
  - 24	MH	Mumbai	273
3. list_df_new[2]
  - 14	TN	Chennai	176
  - 35	TN	Chennai	197
4. list_df_new[3]
  - 15	TN	Chennai	225
  - 24	TN	Chennai	234
  - 26	TN	Chennai	236
  - 29	TN	Chennai	239
  - 36	TN	Chennai	246
  - 37	TN	Chennai	247
  - 38	TN	Chennai	248
  - 47	TN	Chennai	257

In [ ]:
# Check the results.
list_df_yes_48[0]

In [ ]:
# Check the results.
list_df_yes_48[1]

In [ ]:
# Check the results.
list_df_yes_48[2]

In [ ]:
# Check the results.
list_df_yes_48[3]

## DF List > Single DF
- まだ問題が残っていることを重々承知の上で、これを放置して次に進みます（👈急いては事を仕損じるタイプ）。
1. df_non_48 を修正した list_df_yes_48 と、取り分けて温存しておいた df_48 とを合体して復元。
2. 復元した df_list_tmp の中身（108 個の DF）を 1 本の単独 DF に concatenate する（つなげる）。
3. これまでの操作を加えた後で 'state_code' が NaN の row は row ごと drop しても大丈夫なので、dropna しておく。
4. Row を drop したあとは reset_index() して気分一新。

In [ ]:
# 1. Reconstruct the DF list.
df_list_tmp = list_df_yes_48 + list_df_48 
# 2. Concatenate 108 DFs to produce intermediate DF.
df_all_intermed = pd.concat(df_list_tmp)
# 3. Remove garvage rows.
df_all_intermed.dropna(subset = ['state_code'], inplace=True)
# 4. Reset the index of the DF.
df_all_intermed.reset_index(drop=True, inplace=True)

# 中間レビュー Review
- 中間産物の describe() を取って、仕上がり具合を確認します（TABLE 5）。
1. オリジナル PDF の表は複数ページに分かれており、それぞれに表頭が付いています（Microsoft Word でいうと「タイトル行の繰り返し」状態）。このため、df_all_intermed には余分の表頭が含まれています。
  - その結果、例えば、TABLE 5 で state_code （基礎知識: インドの州の略記。英字 2 文字。本資料作成時点で 33 種類）の unique が 70 （本来なら 33 のはず）となっているのは、state_code のところに表頭の文字列が入ってしまっているためと推測されます（head() method などを使って簡単に確認できます）。
2. com_in （現地拠点）の count が 5106 となっているのは良い知らせです（オリジナル PDF の最初に書いてあるように、日系企業の調査時点における現地拠点数は 5102 か所です）。
3. 'sc_num' column（州ごとに振ってある拠点番号。当然 integer でなければならない）が decimal ぽく表示されていますが、DF を直接確認したら、ちゃんと int 型になっていたので大丈夫😊

In [ ]:
# TABLE 5: Description of the intermediate DF
df_all_intermed.describe()

,state_code,location,sc_num,com_in,com_jp,biz_type
count,5142,5108,5106.0,5106,4942,5106
unique,70,737,1034.0,1723,1100,57
top,MH,Gurugram,1.0,Reliance Nippon Life Insurance Company,日本生命保険,26 金融業、保険業/ Finance and insurance
freq,810,456,27.0,628,629,1419


- 'state_code' column に 英字2文字の state code 以外のものが入ってる rows は簡単に drop することができるので、いまは Review 中ですがちょっと作業をして、サクッと drop しておくことにします（👈計画性の欠如）。
- 結果（TABLE 6）を見ると、'state_code' の unique もちゃんと 33 になったし、
- 他の columns も概ね 5102（資料作成時点の現地拠点数）と一致しており、ゴールは近い！

In [ ]:
for i in range(len(df_all_intermed) - 1):
  if not len(df_all_intermed.at[i, 'state_code']) == 2:
    df_all_intermed.drop(i, inplace=True)
# Don't forget to reset the index.
df_all_intermed.reset_index(drop=True, inplace=True)

In [ ]:
# TABLE 6: Description of the intermediate DF
df_all_intermed.describe()

,state_code,location,sc_num,com_in,com_jp,biz_type
count,5102,5102,5102.0,5102,4938,5102
unique,33,734,1033.0,1722,1099,56
top,MH,Gurugram,1.0,Reliance Nippon Life Insurance Company,日本生命保険,26 金融業、保険業/ Finance and insurance
freq,810,456,27.0,628,629,1419


# ☕︎ここで休憩
- ここまでの作業結果を、CSV に保存します（Google Drive に作った data フォルダの中に 'df_all_intermed.csv'という 名前で保管します。）

In [ ]:
# 以下で説明する問題が発覚したため、いったんコメントアウトします。
#df_all_intermed.to_csv(os.path.join(DATA_PATH, 'df_all_intermed.csv'), index=False)

## 重大事実の発覚😱
- Google Drive から df_all_intermed.csv を local に落として、適当なソフトで開いてみると…。
  - イイね、イイね、完璧じゃん、ほぼ出来上がり😸
  - …と思いきや、途中から変なところで row が折り返され、shape がメチャメチャになってしまう事案が多発？なんなの、コレ？😱
  - こりゃ何か変な事をやらかしちゃったかな？と、script 修正を数回試みるも症状は変わらず😭

## 原因及び対策
- しばし茫然自失としていたら、どこからか神様の声が…。
- 「r だよ、r. \r, r’\r’, CR !」
- ああ、思い出した。自分でも「Column Names の確認」のところで、「Column name に全角文字やスペース、改行文字（\r）などが含まれている。」って書いてるじゃん？
- そこで問題の箇所を確認すると、原稿の Excel で「セル内改行」が行われていたとおぼしきところに \r が挿入されていました。
- というわけで、やり直し。改行文字（\r）を差し障りのなさそうな文字列 '\<br\>' に replace してから保存することにします。

In [ ]:
# regex で \r を <br> に replace
df_all_intermed.replace(r'\r', r'<br>', inplace=True, regex=True)
# CSV に保存
df_all_intermed.to_csv(os.path.join(DATA_PATH, 'df_all_intermed.csv'), index=False)

## Excel で保存したら大丈夫だった
- もしかしたら、と思って、CSV じゃなくて Excel なら Microsoft 同士だから忖度してくれるんじゃないか、と思って、 を <br> に replace する前の DF で試したみたところ、
- 予想に違わず、\r が混入した DF も何の問題もなく期待した形式の表として Excel Book に保存できました（ヤレヤレ…）。
- （追記）が、このファイルには依然として地雷が埋まっているわけですから、実際には使わないことにします。

In [ ]:
#df_all_intermed.to_excel(os.path.join(DATA_PATH, 'df_all_intermed.xlsx'), index=False)

# 次回計画
1. 宿題（cf.「結果確認」）を片付ける。それから、CSV におとしたら 'sc_num' はやはり decimal っぽくなっていたのでこれも直す。
2. 'biz-type' の正規化。現状で unique 値が 56 個ですが、これは本当の数の倍ぐらいに増えちゃっています（triad 退治などの影響）
3. 表全体の目視確認
4. 正規化: 'com_in', 'com_jp', 'biz_type' は別の table に分けて foreigh key で引っ張ってくるようにした方が良さそう（言うのは簡単）。